In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sherpa_ai.memory import Belief
from states import get_actions, add_state_machine
from utils import load_processed_dataset
import random
from langchain_openai import ChatOpenAI
from sherpa_ai.agents.qa_agent import QAAgent
from sherpa_ai.events import Event, EventType
from policy import ReactPolicy
from tqdm.notebook import tqdm
import pandas as pd
from datasets import load_dataset


2024-11-19 19:34:44.205 | WARNING  | sherpa_ai.config:<module>:128 - Config: Slack environment variables not set
2024-11-19 19:34:44.206 | INFO     | sherpa_ai.config:<module>:135 - Config: OpenAI environment variables are set
2024-11-19 19:34:44.206 | WARNING  | sherpa_ai.config:check_vectordb_setting:114 - Config: No vector database environment variables are set. Using in-memory Chroma database. This may not be what you intended.
c:\Users\chenp\anaconda3\envs\sherpa\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\chenp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
dataset = load_dataset("Dogdays/clevr_subset", token=True)["train"]

In [43]:
model_name = "gpt-4o-mini"
llm = ChatOpenAI(model=model_name, temperature=0.01)

Next: Run the react state machine with Clevr example

In [44]:
STATE_MACHINE = """
Start --> Filtering: start
Filtering --> Filtering: filter_with_attribute
Filtering --> Querying: continue
Filtering --> Finish: answer
Querying --> Relating: continue
Querying --> Querying: query_attribute
Querying --> Finish: answer
Relating --> Checking: continue
Relating --> Relating: get_related_objects
Relating --> Finish: answer
Checking --> Filtering: continue
Checking --> Checking: get_same_objects
Checking --> Finish: answer
"""

In [45]:
dataset[0]["question"]

"How many metal objects would there be if you didn't include spheres?"

In [46]:
# select a subset of the hugginface dataset
dataset_sub = dataset[:100]

In [47]:
add_state_machine(belief, action_map,  True)

---
State Machine
---
stateDiagram-v2
  direction LR
  classDef s_default fill:white,color:black
  classDef s_inactive fill:white,color:black
  classDef s_parallel color:black,fill:white
  classDef s_active color:red,fill:darksalmon
  classDef s_previous color:blue,fill:azure
  
  state "Start" as Start
  Class Start s_active
  state "Filtering" as Filtering
  Class Filtering s_default
  state "Checking" as Checking
  Class Checking s_default
  state "Relating" as Relating
  Class Relating s_default
  state "Querying" as Querying
  Class Querying s_default
  state "Finish" as Finish
  Class Finish s_default
  
  Start --> Filtering: start
  Filtering --> Filtering: filter_with_attribute
  Filtering --> Querying: other_options
  Filtering --> Finish: answer
  Querying --> Relating: other_options
  Querying --> Querying: query_attribute
  Querying --> Finish: answer
  Relating --> Checking: other_options
  Relating --> Relating: get_related_objects
  Relating --> Finish: answer
  Checkin

In [ ]:
results = []

AGENT_DESCRIPTION = """
You are a question answering assistant helping users to find answers to their questions based on a specific scene.
Each object in the scene contain the following properties: color, size, shape, material, and a unique identifier.
The properties are from a fixed set of values:
– Size: One of large or small.
– Color: One of gray, red, blue, green, brown, purple, cyan, or yellow.
– Shape: One of cube (block), sphere, or cylinder.
– Material: One of rubber (matte) or metal (shinning).
- Unique identifier: The index of the object in the scene, starting from 0.

Scene: {scene}

Objects in the scene also have the following relationships: left, right, front or behind.

Given the question, first identify ALL relevant objects in the scene using filter. Then identify their relations.

If answering the question requires and object that does not exist in the scene, give answer "no" if it is a boolean question, or "0" if it is count question.

When provide action arguments, ONLY use the values from the fixed set of values above.
"""
idx = 0
use_scene = True

for sample in tqdm(dataset):
    scene = sample["scene"]
    test_question = sample["question"]


    belief = Belief()
    action_map = get_actions(belief, llm)
    add_state_machine(belief, action_map, print_sm=False)
    
    belief.set("scene", scene)

    if use_scene:
        belief.set("agent_scene", scene)

    agent_description = AGENT_DESCRIPTION
    
    policy = ReactPolicy(
        role_description=agent_description,
        llm=llm,
    )

    agent = QAAgent(
        llm=llm,
        belief=belief,
        description=agent_description,
        num_runs=10,
        policy=policy,
    )

    belief.set_current_task(
        Event(
            EventType.task, "user", f"{test_question}."
        )
    )
    agent.run()

    if belief.state_machine.state != "Finish":
        belief.state_machine.answer()

    result = [str(event) for event in belief.internal_events]

    answer = belief.get(
        "answer_count_action",
        belief.get(
            "answer_judging_action",
            belief.get(
                "answer_querying_action",
                belief.get("answer_action", "No answer found."),
            ),
        ),
    )

    results.append((result, answer))

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [75]:
dataset[idx]["question"]

"How many metal objects would there be if you didn't include spheres?"

In [76]:
dataset[idx]["answer"]

'2'

In [82]:
results

[(['start: EventType.action - Action: start starts, Args: {}',
   'start: EventType.action_output - Action: start finishes, Observation: There are 8 object: [0, 1, 2, 3, 4, 5, 6, 7]',
   "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'attributes': {'material': 'metal', 'shape': 'cube'}}",
   'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: [0]',
   'answer: EventType.action - Action: answer starts, Args: {}',
   "answer: EventType.action_output - Action: answer finishes, Observation: {'answer': 1}"],
  {'answer': 1}),
 (['start: EventType.action - Action: start starts, Args: {}',
   'start: EventType.action_output - Action: start finishes, Observation: There are 10 object: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]',
   "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'attributes': {'color': 'green', 'shape': 'cube'}}",
   'filter_with_attribute: EventType.action_ou

In [77]:
results

[(['start: EventType.action - Action: start starts, Args: {}',
   'start: EventType.action_output - Action: start finishes, Observation: There are 8 object: [0, 1, 2, 3, 4, 5, 6, 7]',
   "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'attributes': {'material': 'metal', 'shape': 'cube'}}",
   'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: [0]',
   'answer: EventType.action - Action: answer starts, Args: {}',
   "answer: EventType.action_output - Action: answer finishes, Observation: {'answer': 2}"],
  {'answer': 2})]

In [88]:
results[2]

(['start: EventType.action - Action: start starts, Args: {}',
  'start: EventType.action_output - Action: start finishes, Observation: There are 9 object: [0, 1, 2, 3, 4, 5, 6, 7, 8]',
  "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'attributes': {'shape': 'cylinder'}}",
  'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: [0, 1, 3, 4, 5, 6]',
  "filter_with_attribute: EventType.action - Action: filter_with_attribute starts, Args: {'attributes': {'shape': 'cube'}}",
  'filter_with_attribute: EventType.action_output - Action: filter_with_attribute finishes, Observation: [7, 8]',
  'answer: EventType.action - Action: answer starts, Args: {}',
  "answer: EventType.action_output - Action: answer finishes, Observation: {'answer': 8}"],
 {'answer': 8})

In [89]:
real_answers[2]

'8'

In [84]:
real_answers = [question["answer"] for question in dataset]

In [90]:
correct = [1 if str(p[1]["answer"]) == str(a) else 0 for p, a in zip(results, real_answers)]
print(len(correct), sum(correct), sum(correct) / len(correct))

100 65 0.65


In [91]:
df = pd.DataFrame({
    "predicted": [r[1]["answer"] for r in results],
    "reasoning": [r[0] for r in results],
    "actual": real_answers
})

df.to_csv(f"routing_results_{model_name}_no_scene.csv", index=False)

In [60]:
questions[83]

{'split': 'val',
 'image_filename': 'CLEVR_val_007515.png',
 'answer': 'cylinder',
 'question': 'What shape do three large objects have in common?',
 'image_index': 7515}

In [9]:
results_dict = []
for log, question in results:
    results_dict.append({
        "log": log,
        "question": question
    })

df = pd.DataFrame(results_dict)
df.to_csv("results_sm_gpt-4o.csv", index=False)

In [42]:
scenes[2210]

{'objects': [{'color': 'purple',
   'size': 'large',
   'shape': 'cylinder',
   'material': 'metal'},
  {'color': 'green',
   'size': 'large',
   'shape': 'cylinder',
   'material': 'rubber'},
  {'color': 'gray', 'size': 'large', 'shape': 'cube', 'material': 'metal'},
  {'color': 'brown',
   'size': 'small',
   'shape': 'cylinder',
   'material': 'rubber'},
  {'color': 'green',
   'size': 'small',
   'shape': 'cylinder',
   'material': 'metal'}],
 'relationships': {'right': [[], [0, 4], [0, 1, 4], [0, 1, 2, 4], [0]],
  'behind': [[], [0, 2, 3], [0], [0, 2], [0, 1, 2, 3]],
  'front': [[1, 2, 3, 4], [4], [1, 3, 4], [1, 4], []],
  'left': [[1, 2, 3, 4], [2, 3], [3], [], [1, 2, 3]]}}

In [65]:
print(test_question['question'])

Are there an equal number of shiny and matte objects?


In [66]:
test_question

{'split': 'val',
 'image_filename': 'CLEVR_val_007329.png',
 'answer': 'yes',
 'question': 'Are there an equal number of shiny and matte objects?',
 'image_index': 7329}